In [1]:
import os
os.chdir("../")
%pwd

'/home/megh/AI/Projects/Grapevine_Leaf_Classification'

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

2023-09-08 12:20:35.712621: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-08 12:20:35.928613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-08 12:20:35.928993: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
from dataclasses import dataclass
from pathlib import Path   

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int


In [8]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories, save_json   

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path= CONFIG_FILE_PATH,
                 params_file_path= PARAMS_FILE_PATH):

                 self.config = read_yaml(config_file_path)
                 self.params = read_yaml(params_file_path)

                 create_directories([self.config.artifacts_root])


    def get_validation_config(self)-> EvaluationConfig :
            eval_config = EvaluationConfig(
                    
                    path_of_model="artifacts/training/model.h5",
                    training_data = "artifacts/data_ingestion/Grapevine_Leaves_Image_Dataset",
                    all_params= self.params,
                    params_image_size= self.params.IMAGE_SIZE,
                    params_batch_size = self.params.BATCH_SIZE
                                    )
            return eval_config

In [10]:
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [12]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-09-08 13:29:34,077: INFO: common: yaml fileconfig/config.yamlloaded Successfully]
[2023-09-08 13:29:34,081: INFO: common: yaml fileparams.yamlloaded Successfully]
[2023-09-08 13:29:34,081: INFO: common: created directory at:artifacts]
Found 150 images belonging to 5 classes.


2023-09-08 13:29:35.011580: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-09-08 13:29:35.190510: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


10/10 [==============================] - 5s 197ms/step - loss: 20.3902 - accuracy: 0.2000
[2023-09-08 13:29:39,382: INFO: common: json file saved Successfully at scores.json]
